In [ ]:
import json
import requests
from datetime import datetime
from typing import Dict, List
from dateutil.relativedelta import relativedelta
import boto3


In [ ]:

def get_taxi_data(formatted_datetime: str) -> List:
    """
    Retrieves taxi trip data for a specific date from the City of Chicago open data API.

    Parameters
    ----------
    formatted_datetime: str
       The date in 'YYYY-MM-DD' format.

    Returns
    -------
    List
        A list of dictionaries containing taxi trip records.
    """
    
    taxi_url = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"
    response = requests.get(taxi_url)
    taxi_data = response.json()

    return taxi_data

def get_weather_data(formatted_datetime: str) -> Dict:
    """
    Retrieves hourly weather data using the Open-Meteo API.

    Parameters
    ----------
    formatted_datetime : str
        The date in 'YYYY-MM-DD' format.

    Returns
    -------
    Dict
        A dictionary containing hourly weather data (temperature, wind speed, rain, and precipitation).
    """   
    weather_url = 'https://archive-api.open-meteo.com/v1/era5?'
    params = {
        "latitude": 41.85,
        "longitude": -87.65,
        "start_date": formatted_datetime,
        "end_date": formatted_datetime,
        "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
    }

    response = requests.get(weather_url, params=params)
    weather_data = response.json()

    return weather_data

def upload_to_s3(data: Dict, folder_name:str, file_name: str) -> None:
    """
    Upload data to an S3 bucket.

    Parameters
    ----------
    data : dict
        The data to be uploaded.
    folder_name : str
        The name of the folder within the S3 path.
    file_name : str
        The name of the file to be created in the S3 bucket.

    Returns
    -------
        None
    """
    client = boto3.client('s3')
    client.put_object(
        Bucket = "de-chicago-taxi",
        Key = f"raw_data/to_processed/{folder_name}/{file_name}",
        Body = json.dumps(data)
    )

def lambda_handler(event, context):
    current_datetime = datetime.now() - relativedelta(months=2)

    formatted_datetime = current_datetime.strftime('%Y-%m-%d')
    formatted_datetime

    taxi_data_api_call = get_taxi_data(formatted_datetime)

    weather_data_api_call = get_weather_data(formatted_datetime)

    file_name_taxi = f'taxi_raw_{formatted_datetime}.json'
    file_name_weather = f'weather_raw_{formatted_datetime}.json'

    upload_to_s3(data=taxi_data_api_call, folder_name = 'taxi_data', file_name= file_name_taxi)

    upload_to_s3(data=weather_data_api_call, folder_name = 'weather_data', file_name= file_name_weather)


    print('Taxi data has been uploaded to S3!')
   
    print('Weather data has been uploaded to S3!')